In [1]:
!pip install imblearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import imblearn
import numpy as np

In [3]:
with open('drive/MyDrive/Diplom/test4.csv') as f:
    test = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/train4.csv') as f:
    train = pd.read_csv(f,sep='\t')
with open('drive/MyDrive/Diplom/results.csv') as f:
    results = pd.read_csv(f)

In [6]:
results

,features,f1_score_macro,f1_score_micro,f1_score_weighted,balanced_accuracy_score,neg_log_loss,roc_auc_score ovr,roc_auc_score ovo
0,"ups,score,body_length,count",0.4429,0.6345,0.6987,0.6068,-1.0515,0.7838,0.7873
1,+TF-IDF+pol_and_sub,0.5090,0.7135,0.7629,0.6574,-0.8013,0.8654,0.8577
2,+LDA,0.5470,0.7514,0.7905,0.6674,-0.7302,0.8852,0.8760
3,GS_RF,0.5504,0.8671,0.8551,0.5653,-0.3160,0.9523,0.9201
4,GS_LogReg,0.5733,0.7753,0.8073,0.6842,-0.6952,0.9001,0.8888


In [9]:
cols = np.array(train.drop(columns = ['Unnamed: 0', 'Y']).columns)

In [10]:
cols

array(['a3', 'b3', 'c3', 'd20', 'e20', 'f20', 'g20', 'h20', 'i20', 'j20',
       'k20', 'l20', 'm20', 'n20', 'o20', 'p20', 'q20', 'r20', 's20',
       't20', 'u20', 'v20', 'w20', 'ups', 'score', 'body_length', 'Count',
       'album', 'archive', 'article', 'bot created', 'case', 'compose',
       'conspiracy', 'contact', 'created', 'created user', 'day', 'en',
       'en wikipedia', 'faq', 'first', 'fitbamob', 'friendly',
       'friendly version', 'gfycat', 'gif', 'github', 'go', 'good',
       'google', 'hello', 'help', 'hi', 'hi bot', 'html', 'http www',
       'image', 'imgur', 'imgur mirror', 'info', 'jpg', 'know', 'link',
       'linked', 'made', 'make', 'meme', 'message', 'message compose',
       'mirror', 'mobile', 'mobile friendly', 'mobile version', 'name',
       'need', 'new', 'offsided', 'one', 'org', 'org wiki', 'original',
       'pdf', 'people', 'play', 'please', 'please contact', 'point',
       'post', 'posted', 'problem', 'question', 'reddit message',
       'reddit

In [11]:
X_train = train[cols]
y_train = train.Y
X_test = test[cols]
y_test = test.Y

In [12]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, log_loss, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

def metrics(y_test, y_pred, y_pred_proba, name):
  
  #conf = confusion_matrix(y_test, y_pred)
  #print(conf)
  f1_ma = round(f1_score(y_test, y_pred, average='macro'), 4)
  f1_mi = round(f1_score(y_test, y_pred, average='micro'), 4)
  f1_we = round(f1_score(y_test, y_pred, average='weighted'), 4)
  b_acc = round(balanced_accuracy_score(y_test, y_pred), 4)
  neg_ll = round(- log_loss(y_test, y_pred_proba), 4)
  roc_auc_ovr = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovr'), 4)
  roc_auc_ovo = round(roc_auc_score(y_test, y_pred_proba, average='weighted', multi_class = 'ovo'), 4)

  return pd.DataFrame({'features': [name],'f1_score_macro': [f1_ma], 'f1_score_micro': [f1_mi],'f1_score_weighted': [f1_we], 'balanced_accuracy_score': [b_acc], 'neg_log_loss': [neg_ll],'roc_auc_score ovr': [roc_auc_ovr], 'roc_auc_score ovo': [roc_auc_ovo]})


In [13]:
pipel = Pipeline(steps = [('scaler', StandardScaler()), ('model',LogisticRegression(C = 4.6415888336127775, max_iter = 10000, solver = 'newton-cg', class_weight = 'balanced'))])

In [14]:
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, EditedNearestNeighbours

for sampler in [RandomOverSampler(), SMOTE(), ADASYN()]:
  X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
  pipel.fit(X_resampled, y_resampled)
  y_pred = pipel.predict(X_test)
  y_pred_proba = pipel.predict_proba(X_test)
  res = metrics(y_test, y_pred, y_pred_proba, sampler.__class__.__name__)
  results = pd.concat([results, res])

In [15]:
for sampler in [RandomUnderSampler(), NearMiss(version=1),NearMiss(version=2),TomekLinks(),EditedNearestNeighbours()]:
  X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
  pipel.fit(X_resampled, y_resampled)
  y_pred = pipel.predict(X_test)
  y_pred_proba = pipel.predict_proba(X_test)
  res = metrics(y_test, y_pred, y_pred_proba, sampler.__class__.__name__)
  results = pd.concat([results, res])

In [16]:
results

,features,f1_score_macro,f1_score_micro,f1_score_weighted,balanced_accuracy_score,neg_log_loss,roc_auc_score ovr,roc_auc_score ovo
0,"ups,score,body_length,count",0.4429,0.6345,0.6987,0.6068,-1.0515,0.7838,0.7873
1,+TF-IDF+pol_and_sub,0.5090,0.7135,0.7629,0.6574,-0.8013,0.8654,0.8577
2,+LDA,0.5470,0.7514,0.7905,0.6674,-0.7302,0.8852,0.8760
3,GS_RF,0.5504,0.8671,0.8551,0.5653,-0.3160,0.9523,0.9201
4,GS_LogReg,0.5733,0.7753,0.8073,0.6842,-0.6952,0.9001,0.8888
0,RandomOverSampler,0.5781,0.7837,0.8063,0.6399,-0.7583,0.8988,0.8849
0,SMOTE,0.5836,0.7872,0.8123,0.6592,-0.7157,0.9051,0.8878
0,ADASYN,0.5858,0.7900,0.8122,0.6497,-0.7157,0.9072,0.8894
0,RandomUnderSampler,0.5555,0.7681,0.7980,0.6520,-0.7142,0.8913,0.8763
0,NearMiss,0.2705,0.2607,0.2812,0.3380,-9.3130,0.5017,0.4808


In [17]:
results.to_csv('drive/MyDrive/Diplom/results.csv')